In [41]:
import pandas as pd
import ast
import json
import base64
import numpy as np
import os
np.set_printoptions(suppress=True, precision=4)
from scipy import spatial
import numpy as np
import yaml
import os.path as op
from tqdm import tqdm

tqdm.pandas()

### Features Detection

In [42]:
sg_tsv = './features_detection/training/train.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows=range(0,20000), nrows=10000)
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})

In [43]:
sg_tsv = './features_detection/training/train.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows=range(0,20000), nrows=10000)
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
#df_label_detection

In [44]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
full_detection

,id,features,label
0,4679648220,{'features': 'AAAAAAAAAAAAAAAA8JoNP85JNT4AAAAA...,"[{'class': 'Belt', 'conf': 0.9678944945335388,..."
1,4679736464,{'features': 'AAAAAAAAAAAAAAAAhkdEPwAAAABIoKs+...,"[{'class': 'Face', 'conf': 0.923794686794281, ..."
2,4679761535,{'features': 'WYDnPwAAAAAAAAAAsZG9QHhGVz0AAAAA...,"[{'class': 'Dress', 'conf': 0.9395199418067932..."
3,4679771579,{'features': 'AAAAAFNwhDwAAAAAjzkOQQAAAAB5x4Q/...,"[{'class': 'Train', 'conf': 0.9594755172729492..."
4,4679881654,{'features': 'AAAAAAAAAAAAAAAACVxNPQAAAAB6Y/w9...,"[{'class': 'Belt', 'conf': 0.9516404867172241,..."
...,...,...,...
8995,997338199,{'features': 's9yMPgAAAAAAAAAAWzh9P5uqOEAAAAAA...,"[{'class': 'Hair', 'conf': 0.7632176876068115,..."
8996,997722733,{'features': 'N7IGQElTrj0AAAAAxUgmPkTlSD8AAAAA...,"[{'class': 'Man', 'conf': 0.9461695551872253, ..."
8997,997876722,{'features': 'AAAAAF9gpD8AAAAAubgjPQAAAAAAAAAA...,"[{'class': 'Tomato', 'conf': 0.910000860691070..."
8998,99804383,{'features': 'iGkyPi+yeT0AAAAAMgceQHxPhDwAAAAA...,"[{'class': 'Shirt', 'conf': 0.9361402988433838..."


In [ ]:
#df_split = np.array_split(full_detection, 3)

In [45]:
#full_detection_1 = df_split[2]
full_detection_1 = full_detection

In [46]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection_1.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

9000it [01:36, 93.05it/s] 


### Features Segmentation

In [47]:
sg_tsv = './features_test1/training/train.feature.tsv'
df_feature_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows=range(0,20000), nrows=10000)
df_feature_segmentation = df_feature_segmentation.rename(columns={0: "id", 1: "features"})

In [48]:
sg_tsv = './features_test1/training/train.label.tsv'
df_label_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows=range(0,20000), nrows=10000)
df_label_segmentation = df_label_segmentation.rename(columns={0: "id", 1: "label"})
#df_label_segmentation

In [49]:
full_segmentation = pd.merge(df_feature_segmentation, df_label_segmentation, on="id")
full_segmentation

,id,features,label
0,4679648220,{'features': 'V9OhP+JB4UCWO41Ban4SQRZjtEAAAAAA...,"[{'class': 'Bicycle', 'conf': 0.97119373083114..."
1,4679736464,{'features': '4jMaQbnrkEHZc3JAmLC7P0mkbkEAAAAA...,"[{'class': 'Life jacket', 'conf': 0.9988144636..."
2,4679761535,{'features': 'wA33QL8ifkHEDVRAQXEOQRc/UkB0TSFA...,"[{'class': 'Dress', 'conf': 0.975189208984375,..."
3,4679771579,{'features': 'A3zNQMSSyEBsZfU+n5+yQChyskBOZgw/...,"[{'class': 'Skirt', 'conf': 0.9394164085388184..."
4,4679881654,{'features': 'NvofQf5EQUBYbn9B3oalQIritD8Naes/...,"[{'class': 'Flag', 'conf': 0.8809290528297424,..."
...,...,...,...
8995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
8996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
8997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
8998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [50]:
#df_split_seg = np.array_split(full_segmentation, 3)

In [51]:
#full_segmentation_1 = df_split_seg[2]
full_segmentation_1 = full_segmentation

In [52]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_segmentation_1.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

9000it [01:05, 136.67it/s]


### Union

In [53]:
full_not_present = []
for i in tqdm(range(len(full_detection_1))):    
    not_present = []
    for row_dect in full_detection_1.iloc[i]["label"]:
        check = False
        for row_seg in full_segmentation_1.iloc[i]["label"]:
            if 1 - spatial.distance.cosine(row_seg["rect"], row_dect["rect"]) > 0.985:
                check = True
                break
        if not check:
            not_present.append(row_dect)
    full_not_present.append(not_present)

100%|██████████████████████████████████████████████████████████████████████████████| 9000/9000 [08:09<00:00, 18.40it/s]


In [54]:
for elem in full_detection_1["id"].values == full_segmentation_1["id"].values:
    if not elem: 
        print("noooo")

In [55]:
full_segmentation_1["not_present"] = full_not_present

In [56]:
full_segmentation_1["union"] = full_segmentation_1["label"] + full_segmentation_1["not_present"]
full_segmentation_1

,id,features,label,not_present,union
0,4679648220,{'features': 'V9OhP+JB4UCWO41Ban4SQRZjtEAAAAAA...,"[{'class': 'Bicycle', 'conf': 0.97119373083114...","[{'class': 'Belt', 'conf': 0.9678944945335388,...","[{'class': 'Bicycle', 'conf': 0.97119373083114..."
1,4679736464,{'features': '4jMaQbnrkEHZc3JAmLC7P0mkbkEAAAAA...,"[{'class': 'Life jacket', 'conf': 0.9988144636...","[{'class': 'Water', 'conf': 0.6539292335510254...","[{'class': 'Life jacket', 'conf': 0.9988144636..."
2,4679761535,{'features': 'wA33QL8ifkHEDVRAQXEOQRc/UkB0TSFA...,"[{'class': 'Dress', 'conf': 0.975189208984375,...","[{'class': 'Shadow', 'conf': 0.884247660636901...","[{'class': 'Dress', 'conf': 0.975189208984375,..."
3,4679771579,{'features': 'A3zNQMSSyEBsZfU+n5+yQChyskBOZgw/...,"[{'class': 'Skirt', 'conf': 0.9394164085388184...","[{'class': 'Window', 'conf': 0.949151456356048...","[{'class': 'Skirt', 'conf': 0.9394164085388184..."
4,4679881654,{'features': 'NvofQf5EQUBYbn9B3oalQIritD8Naes/...,"[{'class': 'Flag', 'conf': 0.8809290528297424,...","[{'class': 'Tower', 'conf': 0.8458567261695862...","[{'class': 'Flag', 'conf': 0.8809290528297424,..."
...,...,...,...,...,...
8995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...,"[{'class': 'Curtain', 'conf': 0.80887413024902...","[{'class': 'Wall', 'conf': 0.7054150104522705,...","[{'class': 'Curtain', 'conf': 0.80887413024902..."
8996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...,"[{'class': 'Sandal', 'conf': 0.717478156089782...","[{'class': 'Hair', 'conf': 0.7762690186500549,...","[{'class': 'Sandal', 'conf': 0.717478156089782..."
8997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...,"[{'class': 'Apple', 'conf': 0.9743674397468567...","[{'class': 'Pant', 'conf': 0.8092154264450073,...","[{'class': 'Apple', 'conf': 0.9743674397468567..."
8998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...,"[{'class': 'Signboard', 'conf': 0.706024229526...","[{'class': 'Shirt', 'conf': 0.9361402988433838...","[{'class': 'Signboard', 'conf': 0.706024229526..."


In [57]:
for index, row in tqdm(full_segmentation_1.iterrows()):
    if(len(row["label"]) + len(row["not_present"]) != len(row["union"])):
        print("!!!!!!!!!!!!!!!")

9000it [00:02, 4278.69it/s]


In [58]:
sg_tsv = './training_union/train.feature.tsv'
df_feature_saved = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows=range(0,20000), nrows=10000)
df_feature_saved = df_feature_saved.rename(columns={0: "id", 1: "features"})
df_feature_saved

,id,features
0,4679648220,{'features': 'V9OhP+JB4UCWO41Ban4SQRZjtEAAAAAA...
1,4679736464,{'features': '4jMaQbnrkEHZc3JAmLC7P0mkbkEAAAAA...
2,4679761535,{'features': 'wA33QL8ifkHEDVRAQXEOQRc/UkB0TSFA...
3,4679771579,{'features': 'A3zNQMSSyEBsZfU+n5+yQChyskBOZgw/...
4,4679881654,{'features': 'NvofQf5EQUBYbn9B3oalQIritD8Naes/...
...,...,...
8995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...
8996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...
8997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...
8998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...


In [59]:
sg_tsv = './training_union/train.label.tsv'
df_label_saved = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows=range(0,20000), nrows=10000)
df_label_saved = df_label_saved.rename(columns={0: "id", 1: "label"})
df_label_saved

,id,label
0,4679648220,"[{'class': 'Bicycle', 'conf': 0.97119373083114..."
1,4679736464,"[{'class': 'Life jacket', 'conf': 0.9988144636..."
2,4679761535,"[{'class': 'Dress', 'conf': 0.975189208984375,..."
3,4679771579,"[{'class': 'Skirt', 'conf': 0.9394164085388184..."
4,4679881654,"[{'class': 'Flag', 'conf': 0.8809290528297424,..."
...,...,...
8995,997338199,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
8996,997722733,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
8997,997876722,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
8998,99804383,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [60]:
df_full_saved = pd.merge(df_feature_saved, df_label_saved, on="id")
df_full_saved

,id,features,label
0,4679648220,{'features': 'V9OhP+JB4UCWO41Ban4SQRZjtEAAAAAA...,"[{'class': 'Bicycle', 'conf': 0.97119373083114..."
1,4679736464,{'features': '4jMaQbnrkEHZc3JAmLC7P0mkbkEAAAAA...,"[{'class': 'Life jacket', 'conf': 0.9988144636..."
2,4679761535,{'features': 'wA33QL8ifkHEDVRAQXEOQRc/UkB0TSFA...,"[{'class': 'Dress', 'conf': 0.975189208984375,..."
3,4679771579,{'features': 'A3zNQMSSyEBsZfU+n5+yQChyskBOZgw/...,"[{'class': 'Skirt', 'conf': 0.9394164085388184..."
4,4679881654,{'features': 'NvofQf5EQUBYbn9B3oalQIritD8Naes/...,"[{'class': 'Flag', 'conf': 0.8809290528297424,..."
...,...,...,...
8995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
8996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
8997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
8998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [61]:
#df_full_saved_1 = np.array_split(df_full_saved, 3)
#df_full_saved_1 = df_split_seg[2]
df_full_saved_1 = df_full_saved

In [62]:
bbox_id = 0
for index, row in tqdm(df_full_saved_1.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id += 1

9000it [01:08, 131.32it/s]


In [63]:
# confronto per vedere se sono state salvate correttamente

In [64]:
for elem in df_full_saved_1["id"].values == full_segmentation_1["id"].values:
    if not elem: 
        print("noooo")

In [65]:
full_segmentation_1

,id,features,label,not_present,union
0,4679648220,{'features': 'V9OhP+JB4UCWO41Ban4SQRZjtEAAAAAA...,"[{'class': 'Bicycle', 'conf': 0.97119373083114...","[{'class': 'Belt', 'conf': 0.9678944945335388,...","[{'class': 'Bicycle', 'conf': 0.97119373083114..."
1,4679736464,{'features': '4jMaQbnrkEHZc3JAmLC7P0mkbkEAAAAA...,"[{'class': 'Life jacket', 'conf': 0.9988144636...","[{'class': 'Water', 'conf': 0.6539292335510254...","[{'class': 'Life jacket', 'conf': 0.9988144636..."
2,4679761535,{'features': 'wA33QL8ifkHEDVRAQXEOQRc/UkB0TSFA...,"[{'class': 'Dress', 'conf': 0.975189208984375,...","[{'class': 'Shadow', 'conf': 0.884247660636901...","[{'class': 'Dress', 'conf': 0.975189208984375,..."
3,4679771579,{'features': 'A3zNQMSSyEBsZfU+n5+yQChyskBOZgw/...,"[{'class': 'Skirt', 'conf': 0.9394164085388184...","[{'class': 'Window', 'conf': 0.949151456356048...","[{'class': 'Skirt', 'conf': 0.9394164085388184..."
4,4679881654,{'features': 'NvofQf5EQUBYbn9B3oalQIritD8Naes/...,"[{'class': 'Flag', 'conf': 0.8809290528297424,...","[{'class': 'Tower', 'conf': 0.8458567261695862...","[{'class': 'Flag', 'conf': 0.8809290528297424,..."
...,...,...,...,...,...
8995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...,"[{'class': 'Curtain', 'conf': 0.80887413024902...","[{'class': 'Wall', 'conf': 0.7054150104522705,...","[{'class': 'Curtain', 'conf': 0.80887413024902..."
8996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...,"[{'class': 'Sandal', 'conf': 0.717478156089782...","[{'class': 'Hair', 'conf': 0.7762690186500549,...","[{'class': 'Sandal', 'conf': 0.717478156089782..."
8997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...,"[{'class': 'Apple', 'conf': 0.9743674397468567...","[{'class': 'Pant', 'conf': 0.8092154264450073,...","[{'class': 'Apple', 'conf': 0.9743674397468567..."
8998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...,"[{'class': 'Signboard', 'conf': 0.706024229526...","[{'class': 'Shirt', 'conf': 0.9361402988433838...","[{'class': 'Signboard', 'conf': 0.706024229526..."


In [66]:
for i in tqdm(range(len(df_full_saved_1))):    
    if df_full_saved_1.iloc[i]["id"] == full_segmentation_1.iloc[i]["id"] and len(df_full_saved_1.iloc[i]["label"]) == len(full_segmentation_1.iloc[i]["union"]):
        for j in range(len(df_full_saved_1.iloc[i]["label"])):
            if not (df_full_saved_1.iloc[i]["label"][j]["class"] == full_segmentation_1.iloc[i]["union"][j]["class"] and df_full_saved_1.iloc[i]["label"][j]["rect"] == full_segmentation_1.iloc[i]["union"][j]["rect"] and all(df_full_saved_1.iloc[i]["label"][j]["feature"] == full_segmentation_1.iloc[i]["union"][j]["feature"])):
                print("errore con indice: {}, con elemento in posizione: {}".format(i, j))
    else:
        print("errore con indice: ", i)

100%|██████████████████████████████████████████████████████████████████████████████| 9000/9000 [07:48<00:00, 19.23it/s]
